In [2]:
import pandas as pd, numpy as np
from rapidfuzz import process, fuzz,utils
np.set_printoptions(threshold=np.inf)

In [61]:
ITM_ID_GTIN = pd.read_csv('ITM_ID_GTIN_102523.csv')
pim_colors = pd.read_csv('PIM_Color_Attributes_102523.csv')
pim_scents = pd.read_csv('PIM_Scent_Attributes_102523.csv')

In [62]:
pim_colors = ITM_ID_GTIN.merge(pim_colors, on = 'ITM_ID', how = 'right')
pim_scents = ITM_ID_GTIN.merge(pim_scents, on = 'ITM_ID', how = 'right')

###  COLOR

In [12]:
pim_df = pd.read_csv('PIM_Data_New_50_82Mn.zip', low_memory= True)

In [65]:
pim_colors = pim_colors[pim_colors.COLOR_CD.notnull()]

In [66]:
pim_colors = pim_df.merge(pim_colors[['GTIN_NO','COLOR_CD','COLOR_VLU']], how = 'right', on = 'GTIN_NO')

In [67]:
colors_DPT = pd.DataFrame(pim_colors.groupby('DPT_DSC').apply(lambda x: ','.join(x.COLOR_CD.unique())), columns= ['UNIQUE COLORS']).reset_index()
colors_DPT_DSCs = colors_DPT[~colors_DPT['UNIQUE COLORS'].isin(['NONE','OTHER'])].DPT_DSC.unique()
len(colors_DPT_DSCs)

64

In [68]:
colors_DPT = colors_DPT[colors_DPT.DPT_DSC.isin(colors_DPT_DSCs)].reset_index(drop = True)
colors_DPT['UNIQUE COLOR COUNT'] = colors_DPT['UNIQUE COLORS'].\
apply(lambda x: len(','.join([i for i in x.split(',') if not i in ['NONE','OTHER']]).split(',')))

In [69]:
colors_DPT['ITEM_WITH_COLOR_COUNT'] = colors_DPT.DPT_DSC.map(dict(pim_colors.groupby('DPT_DSC').size()))

In [70]:
colors_DPT['TOTAL_ITEM_COUNT'] = colors_DPT.DPT_DSC.map(dict(pim_df.drop_duplicates('GTIN_NO').groupby('DPT_DSC').size()))

In [71]:
pim_colors.COLOR_CD.value_counts()

MULTI_COLORED      251073
BLACK               87261
BLUE                82926
GRAY                82080
WHITE               65977
                    ...  
FOREVER PINK            1
ICED CAPPUCCINO         1
DEADPOOL                1
SMOKY ROSE              1
ALWAYS APRICOT          1
Name: COLOR_CD, Length: 936, dtype: int64

In [72]:
colors_DPT

,DPT_DSC,UNIQUE COLORS,UNIQUE COLOR COUNT,ITEM_WITH_COLOR_COUNT,TOTAL_ITEM_COUNT
0,ACCESSORIES,"BLACK,RED,BLUE,NONE,OTHER,NAVY,CHARCOAL_GREY,R...",84,1179,36382
1,BABY,"PINK,GOLD,CLEAR,WHITE,ASSORTED,MULTI_COLORED,A...",61,1251,21011
2,BAKERY,"RED,WHITE,PURPLE,TAN,DARK_BROWN,YELLOW,BLONDE,...",72,1114,39647
3,BEAUTY,"BEIGE,BLUE,RED,IVORY,BROWN,PINK,GOLD,TAN,BLACK...",556,12345,277878
4,BEEF,"NONE,BEIGE,BROWN,RED",3,7,23027
...,...,...,...,...,...
59,TOBACCO,"BLUE,CLEAR",2,2,7304
60,TOYS,"MULTI_COLORED,GREEN,YELLOW,BABY_BLUE,ARMY_GREE...",101,43283,183260
61,VEGETABLES,"ORANGE,DARK_ORANGE,ORANGE_RED,MULTI_COLORED,GR...",38,993,16683
62,VIDEO/ELEC/COMP,"OTHER,PINK,NATURAL,CHERRYWOOD,BLACK,CHARCOAL_G...",74,6388,56269


In [91]:
colors_DPT['ITEM_WITH_COLOR_COUNT'].sum()

952268

In [92]:
colors_DPT['TOTAL_ITEM_COUNT'].sum()

4963297

In [75]:
colors_DPT.to_excel('COLORS_SUMMARY_BY_DEPT_102523.xlsx')

###  SCENT

In [76]:
pim_scents = pim_scents[pim_scents.SCENT_CD.notnull()]

In [77]:
pim_scents = pim_df.merge(pim_scents[['GTIN_NO','SCENT_CD','SCENT_VLU']], how = 'right', on = 'GTIN_NO')

In [78]:
scents_DPT = pd.DataFrame(pim_scents.groupby('DPT_DSC').apply(lambda x: ','.join(x.SCENT_CD.unique())), columns= ['UNIQUE SCENTS']).reset_index()
scents_DPT_DSCs = scents_DPT[~scents_DPT['UNIQUE SCENTS'].isin(['NONE','OTHER'])].DPT_DSC.unique()
len(scents_DPT_DSCs)

23

In [79]:
scents_DPT = scents_DPT[scents_DPT.DPT_DSC.isin(scents_DPT_DSCs)].reset_index(drop = True)
scents_DPT['UNIQUE SCENT COUNT'] = scents_DPT['UNIQUE SCENTS'].\
apply(lambda x: len(','.join([i for i in x.split(',') if not i in ['NONE','OTHER']]).split(',')))

In [80]:
scents_DPT['ITEM_WITH_SCENT_COUNT'] = scents_DPT.DPT_DSC.map(dict(pim_scents.groupby('DPT_DSC').size()))

In [81]:
scents_DPT['TOTAL_ITEM_COUNT'] = scents_DPT.DPT_DSC.map(dict(pim_df.drop_duplicates('GTIN_NO').groupby('DPT_DSC').size()))

In [82]:
pim_scents.SCENT_CD.value_counts()

NONE                   3080
OTHER                  1306
UNSCENTED               546
LAVENDAR                165
FRESH                   162
                       ... 
MOROCCAN ROSE             1
PEAR & GREEN TEA          1
SESAME                    1
LAVENDER & ROSEMARY       1
SPICY CHAI                1
Name: SCENT_CD, Length: 1164, dtype: int64

In [83]:
scents_DPT

,DPT_DSC,UNIQUE SCENTS,UNIQUE SCENT COUNT,ITEM_WITH_SCENT_COUNT,TOTAL_ITEM_COUNT
0,BABY,"MEDICATED,FRAGRANCE FREE,UNSCENTED,BABY_FRESH,...",65,240,21011
1,BEAUTY,"COCONUT MILK,MANGO,TEA TREE,TROPICAL BOOST,SAN...",557,2595,277878
2,CANDY,ACAI AND WHITE TEA,1,1,50568
3,CHILDRENS,LAVENDER,1,1,204629
4,CKY/CRKR/SNK,"ACAI,ACAI AND WHITE TEA",2,2,36067
5,FURN/HOME DEC,"NONE,TEAKWOOD,OTHER,HONEYSUCKLE,UNSCENTED,GARD...",134,1555,557632
6,GARDEN-OUTDOOR,"UNSCENTED,NONE",1,4,245563
7,GM-MISC,FRESH,1,1,2006
8,GROC NON-EDIBLE,"ORIGINAL,LINENSKY,APPLE,OTHER,PUMPKIN,MOUNTAIN...",230,949,37702
9,GROC-ALL OTHER,"NONE,FIELD FLOWERS,ACAI & MANDARIN ORANGE,4 SE...",5,39,195187


In [84]:
scents_DPT['ITEM_WITH_SCENT_COUNT'].sum()

9607

In [85]:
scents_DPT['TOTAL_ITEM_COUNT'].sum()

3231920

In [86]:
scents_DPT.to_excel('SCENTS_SUMMARY_BY_DEPT_102523.xlsx')